In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install jovian --upgrade --quiet

In [ ]:
import torch
import torchvision.transforms as t
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import os
from torchvision.utils import make_grid
import torch.nn as nn
import torch.functional as F
import torch.nn.functional as ff
from tqdm.notebook import tqdm

In [ ]:
# Setting up the data directory and loading the dataset
dataset= '../input/flickrfaceshq-dataset-ffhq'
data_dir= '../input'

In [ ]:
print(os.listdir(data_dir)[:15])

In [ ]:
# Normalizing parameter, batch size, image size
stats= (0.5,0.5,0.5), (0.5,0.5,0.5)
batch_size= 150
image_size= 64

In [ ]:
# Appling the transformation and normalization on the image data
image_data= ImageFolder(data_dir, transform= t.Compose([t.Resize(image_size), t.CenterCrop(image_size), t.ToTensor(), t.Normalize(*stats)]))

In [ ]:
# Making data loader from the dataset
imagedata_loader= DataLoader(image_data, batch_size, pin_memory= 4, num_workers= True, shuffle= True)

### Vizualise the images of the dataset

In [ ]:
# Function to normalization
def denorm(image):
    return image * stats[0][1] + stats[0][1]

# Function to make the grid of the images
def grid_frame(image, nums= 70):
    fig, ax= plt.subplots(figsize= (12,10))
    ax.set_xticks([]), ax.set_yticks([])
    ax.imshow(make_grid(denorm(image.detach()[:nums]), nrow= 10).permute(1,2,0))
    
# Function to make the grid of batches
def make_batch(imagedata_loader, nums= 70):
    for image, _ in imagedata_loader:
        grid_frame(image)
        break

In [ ]:
# Example images from the data set
make_batch(imagedata_loader)

### To work with GPUs

In [ ]:
# To clear the cache
torch.cuda.empty_cache()

In [ ]:
# Function to return the available device
def get_device():
    if torch.cuda.is_available():
        return ('cuda')
    else:
        return ('cpu')
    
# Function to move the data on avialable device
def to_device(data, device):
    if isinstance(data, (list, tuple)):
        return [to_device(xdata, device) for xdata in data]
    else:
        return data.to(device, non_blocking= True)
    
# Class to move the batch of data in avialable device
class devicedataloader():
    def __init__(self, imagedata_loader, device):
        self.dl = imagedata_loader
        self.device= device
    
    def __iter__(self):
        for batch in self.dl:
            yield to_device(batch, self.device)
            
    def __len__(self):
        return len(self.dl)

In [ ]:
device= get_device()
device

In [ ]:
torch.cuda.empty_cache()

### Discriminator 
Discriminator used to predict weather it belong to dataset or generated using generated model

In [ ]:
discriminator= nn.Sequential(
    nn.Conv2d(3, 64, kernel_size=4 , stride= 2, padding= 1, bias= False),
    nn.BatchNorm2d(64),
    nn.LeakyReLU(0.2, inplace= True),
    
    nn.Conv2d(64, 128, kernel_size=4 , stride= 2, padding= 1, bias= False),
    nn.BatchNorm2d(128),
    nn.LeakyReLU(0.2, inplace= True),
    
    nn.Conv2d(128, 256, kernel_size=4 , stride= 2, padding= 1, bias= False),
    nn.BatchNorm2d(256),
    nn.LeakyReLU(0.2, inplace= True),
    
    nn.Conv2d(256, 512, kernel_size=4 , stride= 2, padding= 1, bias= False),
    nn.BatchNorm2d(512),
    nn.LeakyReLU(0.2, inplace= True),
    
    nn.Conv2d(512, 1, kernel_size= 4, stride= 1, padding= 0, bias= False),
    
    nn.Flatten(),
    nn.Sigmoid()
)

### Generator Function
Generator is used to generate the image using the image dataset

In [ ]:
# Defining the latent size (Number of images in one batch)
latent_size= 130

In [ ]:
generator= nn.Sequential(
    nn.ConvTranspose2d(latent_size, 512, kernel_size= 4, stride= 1, padding= 0, bias= False),
    nn.BatchNorm2d(512),
    nn.ReLU(inplace= True),
    
    nn.ConvTranspose2d(512, 256, kernel_size= 4, stride= 2, padding= 1, bias= False),
    nn.BatchNorm2d(256),
    nn.ReLU(inplace= True),
    
    nn.ConvTranspose2d(256, 128, kernel_size= 4, stride= 2, padding= 1, bias= False),
    nn.BatchNorm2d(128),
    nn.ReLU(inplace= True),
    
    nn.ConvTranspose2d(128, 64, kernel_size= 4, stride= 2, padding= 1, bias= False),
    nn.BatchNorm2d(64),
    nn.ReLU(inplace= True),
    
    nn.ConvTranspose2d(64, 3, kernel_size= 4, stride= 2, padding= 1, bias= False),
    nn.Tanh()
)

In [ ]:
# Moving data loader to the avialable device
imagedata_loader= devicedataloader(imagedata_loader, device)

In [ ]:
# Testing the genertor model (is it working or not)
xb= torch.randn(batch_size, latent_size, 1 , 1)      # Batch of the images of size latent x 1 x 1
fake_image= generator(xb)           # Creating fake image using generator
grid_frame(fake_image)
print(fake_image.shape)

In [ ]:
# Moving discriminator and generator on the availabel device
to_device(discriminator, device)
to_device(generator, device)

### Training Discriminator and generator 

In [ ]:
def train_dis(images, opt_d):
    
    # Reseting the gradinet
    opt_d.zero_grad()
    
    # Target for the real image is set to be 1 
    real_pred= discriminator(images)
    real_target= torch.ones(images.size(0), 1, device= device)
    real_loss= ff.binary_cross_entropy(real_pred, real_target)
    real_score= torch.mean(real_loss).item()
    
    # Generating the fake images and setting the label as 0
    latent_vector= torch.randn(batch_size, latent_size, 1, 1, device= device)
    fake_image= generator(latent_vector)
    
    fake_image_pred= discriminator(fake_image)
    fake_target= torch.zeros(fake_image.size(0), 1, device= device)
    fake_loss= ff.binary_cross_entropy(fake_image_pred, fake_target)
    fake_score= torch.mean(fake_loss).item()
    
    # Total loss and gradient 
    loss= fake_loss + real_loss
    loss.backward()
    opt_d.step()
    
    return loss.item(), real_score, fake_score

In [ ]:
def train_generator(opt_g):
    opt_g.zero_grad()
    
    latent_vector= torch.randn(batch_size, latent_size, 1, 1, device= device)
    
    fake_images= generator(latent_vector)
    
    # Taking prediction on fake images we generated using generator and setting target as 1
    fake_pred= discriminator(fake_images)
    target= torch.ones(fake_images.size(0), 1, device= device)
    loss= ff.binary_cross_entropy(fake_pred, target)
    
    loss.backward()
    opt_g.step()
    
    return loss.item()

### Defining fit function to train whole model

In [ ]:
def fit(epochs, lr, idx= 1):
    
    # Creating list to track the performance of the model
    losses_gen= []
    losses_dis= []
    real_score= []
    fake_score= []
    
    # Initalizing the optimizer
    opt_d= torch.optim.Adam(discriminator.parameters(), lr, betas= (0.5, 0.9))
    opt_g= torch.optim.Adam(generator.parameters(), lr, betas= (0.5, 0.9))
    
    # Train the model for epochs
    for epoch in range(epochs):
        
        for real_images, _ in tqdm(imagedata_loader):
            loss_d, real_s, fake_s= train_dis(real_images, opt_d)
            loss_g= train_generator(opt_g)
            
        # Storing the losses
        losses_gen.append(loss_g)
        losses_dis.append(loss_d)
        real_score.append(real_s)
        fake_score.append(fake_s)
        
        print('Epochs: [{}], Generator loss: {:.4f}, Discriminator loss: {:.4f}, Real Score: {:.4f}, Fake Score: {:.4f}'.
              format(epoch, loss_g, loss_d, real_s, fake_s))
        
    return losses_gen, losses_dis, real_score, fake_score

### Now saving the progress of our model in the form of image

In [ ]:
from torchvision.utils import save_image

In [ ]:
# Images from which we track the progress of our model
latent_tracker= torch.randn(batch_size, latent_size, 1, 1, device= device)

In [ ]:
# Creating the directory to save all those images which is used to see the progress of the model
tracking_dir= 'generated'
os.makedirs(tracking_dir, exist_ok= True)

In [ ]:
# Defining the function to save the image and also plot the images
def saving(latent, index, show= True):
    fake_image= generator(latent)
    fimage_name= 'generated-images-{0:0=4d}.png'.format(index)
    save_image(denorm(fake_image), os.path.join(tracking_dir, fimage_name), nrow= 10)
    print('Saving', fimage_name)
    if show:
        fig, ax= plt.subplots(figsize= (10,15))
        ax.set_xticks([]), ax.set_yticks([])
        plt.imshow(make_grid(fake_image.cpu().detach(), nrow= 10).permute(1,2,0))

In [ ]:
saving(latent_tracker, 0)

In [ ]:
import jovian
jovian.commit(project= 'humans-faces-gans')

In [ ]:
#Definig hyperparameters
lr= 0.002
epochs= 30

In [ ]:
jovian.log_hyperparams(learning_rate= lr, No_of_epochs= epochs)

In [ ]:
jovian.log_metrics(Batch_size= batch_size, Latent_size= latent_size)

In [ ]:
# lets train the model then we will see the progress
0history= fit(epochs, lr)

In [ ]:
saving(latent_tracker, 1)

In [ ]:
# Plotting the graph to see the change in the loss after each epoch
losses= []
plt.plot()